# MVD 5. cvičení

## 1. část - TF-IDF s word embeddingy

V minulém cvičení bylo za úkol implementovat TF-IDF algoritmus nad datasetem z Kagglu. Dnešní cvičení je rozšířením této úlohy s použitím word embeddingů. Lze použít předtrénované GloVe embeddingy ze 3. cvičení, nebo si v případě zájmu můžete vyzkoušet práci s Word2Vec od Googlu (najdete [zde](https://code.google.com/archive/p/word2vec/)).

Cvičení by mělo obsahovat následující části:
- Načtení článků a embeddingů
- Výpočet document vektorů pomocí TF-IDF a word embeddingů 
    - Pro výpočet TF-IDF využijte [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) z knihovny sklearn
    - Vážený průměr GloVe / Word2Vec vektorů

<center>
$
doc\_vector = \frac{1}{|d|} \sum\limits_{w \in d} TF\_IDF(w) glove(w)
$
</center>

- Dotaz bude transformován stejně jako dokument

- Výpočet relevance pomocí kosinové podobnosti
<center>
$
score(q,d) = cos\_sim(query\_vector, doc\_vector)
$
</center>

### Načtení článků

In [1]:
import spacy
import csv
import math
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
def load_articles(file_path: str) -> tuple:
    with open(file_path, 'rt') as file:
        reader = csv.reader(file, delimiter=',')
        data = [article for article in reader]
        return data[0], data[1:]

header, articles = load_articles('articles.csv')

### Načtení embeddingů

In [4]:
def load_embedded(file_path: str) -> tuple:
    with open(file_path, 'rt') as file:
        words = []
        vectors = []

        for line in file:
            parts = line.strip().split(' ')

            words.append(parts[0])
            vectors.append([float(part) for part in parts[1:]])
    
    word2idx = {word : index for index, word in enumerate(words)}
    
    return words, np.array(vectors), word2idx
    
w, v, w2i = load_embedded('glove.6B.50d.txt')

In [5]:
def adjust_field(field: str, to_be_removed: re.Pattern, lemmatizer) -> str:
    field = field.lower()
    field = re.sub(to_be_removed, '', field)
    field = re.sub('\s', ' ', field)
    
    return " ".join([token.lemma_ for token in lemmatizer(field)])  

### TF-IDF + Word2Vec a vytvoření doc vektorů

In [21]:
vectorizer = TfidfVectorizer()
to_be_removed = r'[.,?!/\\\"`\-:()\[\]*|—’–]'
lemmatizer = spacy.load('en_core_web_sm', disable=['parser', 'ner']) # NLTK

def adjust_articles(articles: list, fields: list) -> list:
    adjusted_articles = []

    for article in articles:
        words = []

        for num in fields:
            modified_field = adjust_field(article[num], to_be_removed, lemmatizer)
            words.extend([word.strip() for word in re.split(r'\s', modified_field) if word != ''])
            
        adjusted_articles.append(words)
        
    return adjusted_articles

def inverted_index(articles: list) -> dict:
    index = {}
    
    for i, article in enumerate(articles):
        for word in article:
            if word in index:
                index[word].append(i)
            else:
                index[word] = [i]
    
    return index

def filter_words(index: dict, glove: list) -> list:
    blacklist = []
    
    index_words = set(index.keys())
    glove_words = set(glove)
    
    blacklist = index_words - glove_words
    
    return blacklist

In [22]:
adj_art = adjust_articles(articles, [5])

In [23]:
index = inverted_index(adj_art)
blacklist = filter_words(index, w)

In [24]:
vectors = np.zeros(v.shape)

for i, article in enumerate(adj_art):
    adj_art[i] = list(set(article) - blacklist)

for i, article in enumerate(adj_art):
    vector = np.zeros([1, v.shape[1]])
    
    for word in article:
            vector += (article.count(word) * math.log((len(adj_art) + 1) / len(index[word]))) * np.array(v[w2i[word]])

    vectors[i, :] = (1 / len(article)) * vector
    

### Transformace dotazu a výpočet relevance

In [25]:
def similarity_one_to_all(a, b):
    return (b @ a).reshape(1, b.shape[0]) / (np.linalg.norm(a) * np.linalg.norm(b, axis=1, keepdims=True)).reshape(1, b.shape[0])

q = 'coursera vs udacity machine learning'

adjusted_query = adjust_articles([[q]], [0])
index = inverted_index(adjusted_query)
blacklist = filter_words(index, w)

query_vect = np.zeros([1, v.shape[1]])

for i, query in enumerate(adjusted_query):
    adjusted_query[i] = list(set(query) - blacklist)

for i, query in enumerate(adjusted_query):
    vector = np.zeros([1, v.shape[1]])

    for word in query:
            vector += (query.count(word) * math.log((len(adjusted_query) + 1) / len(index[word]))) * np.array(v[w2i[word]])

    query_vect[i, :] = (1 / len(query)) * vector

scores = similarity_one_to_all(query_vect.flatten(), vectors).flatten()
order = np.argsort(-scores)

print('{:3}  {:40}     {:40}     {:3.5}'.format('', 'title', 'text', 'scores'))

for i in order[:15]:
    print('{:3}  {:40}...  {:40}...  {:3.5}'.format(i, articles[i][4][0:40], articles[i][5][0:40], scores[i]))

     title                                        text                                         score
260  The $1700 great Deep Learning box: Assem...  Updated April 2018: Uses CUDA 9, cuDNN 7...  0.69648
226  The $1700 great Deep Learning box: Assem...  Updated April 2018: Uses CUDA 9, cuDNN 7...  0.69648
201  The $1700 great Deep Learning box: Assem...  Updated April 2018: Uses CUDA 9, cuDNN 7...  0.69648
149  The $1700 great Deep Learning box: Assem...  Updated April 2018: Uses CUDA 9, cuDNN 7...  0.69648
175  The $1700 great Deep Learning box: Assem...  Updated April 2018: Uses CUDA 9, cuDNN 7...  0.69648
182  Into the Age of Context – Crossing the P...  I spent most of my early career proclaim...  0.68281
 88  Reinforcement Learning from scratch – In...  Want to learn about applied Artificial I...  0.68206
  4  Reinforcement Learning from scratch – In...  Want to learn about applied Artificial I...  0.68206
157  Reinforcement Learning from scratch – In...  Want to learn about appli

/tmp/ipykernel_27723/2257565246.py:2: RuntimeWarning: invalid value encountered in divide
  return (b @ a).reshape(1, b.shape[0]) / (np.linalg.norm(a) * np.linalg.norm(b, axis=1, keepdims=True)).reshape(1, b.shape[0])


## Bonus - Našeptávání

Bonusem dnešního cvičení je našeptávání pomocí rekurentních neuronových sítí. Úkolem je vytvořit jednoduchou rekurentní neuronovou síť, která bude generovat text (character-level přístup). 

Optimální je začít po dokončení cvičení k předmětu ANS, kde se tato úloha řeší. 

Dataset pro učení vaší neuronové sítě naleznete na stránkách [Yahoo research](https://webscope.sandbox.yahoo.com/catalog.php?datatype=l&guccounter=1), lze využít např. i větší [Kaggle dataset](https://www.kaggle.com/c/yandex-personalized-web-search-challenge/data) nebo vyhledat další dataset na [Google DatasetSearch](https://datasetsearch.research.google.com/).

Vstupem bude rozepsaný dotaz a výstupem by měly být alespoň 3 dokončené dotazy.